In [1]:
# Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [33.6 kB]
Get:13 http

In [2]:
# Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

--2025-03-17 07:02:36--  https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228721937 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.1-bin-hadoop3.2.tgz’

spark-3.1.1-bin-had 100%[===================>] 218.13M  12.0MB/s    in 16s     

2025-03-17 07:02:53 (13.3 MB/s) - ‘spark-3.1.1-bin-hadoop3.2.tgz’ saved [228721937/228721937]



In [3]:
# Настройка переменных окружения
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [4]:
# Установка findspark
!pip install -q findspark

In [5]:
# Инициализация findspark
import findspark
findspark.init()

In [6]:
!pip install pyspark


**Проверка установки. Инициализация сессии, импорт необходимых далее библиотем**

In [7]:
# Создание SparkContext (проверка установки)
import re
from pyspark.sql.functions import udf, col, year, lit, when, explode, regexp_replace, lower, split, rank
from typing import List
import pyspark.sql as sql
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import max, sum, desc, countDistinct
import os

spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
    .getOrCreate()

sc = spark.sparkContext

print(sc.version)

3.1.1


**Загрузка данных**

In [8]:
posts_data = spark.read.format('xml').options(rowTag='row').load('posts_sample.xml')
print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [9]:
prog_lang_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv('programming-languages.csv')
print("Programming languages")
prog_lang_data.printSchema()

Programming languages
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



**Выполнение задания**

Задание: Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года. Получившийся отчёт сохранить в формате Apache Parquet.

In [14]:
# Функция для извлечения тегов
def extract_tags(tags_string):
    if tags_string is None:
        return []
    # Удаляем "<" и ">", разделяем по "><" или ","
    tags_string = tags_string.replace("><", ",")
    tags_string = tags_string.replace("<", "")
    tags_string = tags_string.replace(">", "")
    return [tag.strip() for tag in tags_string.split(",")] # Разбиваем и убираем лишние пробелы

# Заменяем None значения в _LastActivityDate на дату по умолчанию
posts_data = posts_data.fillna({'_LastActivityDate': '1900-01-01'})

# Заменяем None значения в _Tags на пустые строки
posts_data = posts_data.withColumn("_Tags",
                                   when(col("_Tags").isNull(), lit("")).otherwise(col("_Tags")))

# Создаем столбец с массивом тегов, используя split и regexp_replace
posts_data = posts_data.withColumn("tags",
    when(col("_Tags").isNull(), lit(None).cast(ArrayType(StringType()))) # Обработка None
    .otherwise(split(regexp_replace(col("_Tags"), "[<>]", ","), ",")) # Замена символов и разделение
)

# Извлекаем год из столбца "_LastActivityDate" и создаем новый столбец "year"
posts_data_simplified = posts_data \
    .withColumn("year", year(col("_LastActivityDate")))

# Выбираем только столбцы "tags", "year" и "_ViewCount", переименовывая "_ViewCount" в "views"
posts_data_simplified = posts_data_simplified.select(
    col("tags"),
    col("year"),
    col("_ViewCount").alias("views")
)

# Получаем первые 10 строк результирующего DataFrame
first_rows = posts_data_simplified.head(10)

# Выводим строки
for i, row in enumerate(first_rows):
    print(i+1, row)

1 Row(tags=['', 'c#', '', 'floating-point', '', 'type-conversion', '', 'double', '', 'decimal', ''], year=2019, views=42817)
2 Row(tags=['', 'html', '', 'css', '', 'internet-explorer-7', ''], year=2019, views=18214)
3 Row(tags=[''], year=2017, views=None)
4 Row(tags=['', 'c#', '', '.net', '', 'datetime', ''], year=2019, views=555183)
5 Row(tags=['', 'c#', '', 'datetime', '', 'time', '', 'datediff', '', 'relative-time-span', ''], year=2019, views=149445)
6 Row(tags=[''], year=2018, views=None)
7 Row(tags=['', 'html', '', 'browser', '', 'timezone', '', 'user-agent', '', 'timezone-offset', ''], year=2019, views=176405)
8 Row(tags=['', '.net', '', 'math', ''], year=2018, views=123231)
9 Row(tags=[''], year=2010, views=None)
10 Row(tags=[''], year=2010, views=None)


In [15]:
# Развертываем массив тегов в отдельные строки
posts_data_simplified = posts_data_simplified.withColumn("tag", explode(col("tags")))

posts_data_simplified = posts_data_simplified.select(
    col("tag"),
    col("year"),
    col("views")
)

# Группировка по году последней активности и тегам, суммирование всех просмотров для каждого языка программирования в пределах одного года
posts_data_sorted = posts_data_simplified.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Убираем строки с пустым тегом
posts_data_sorted = posts_data_sorted.filter(posts_data_sorted["tag"] != "")

# Сортировка по году и убыванию total_views
posts_data_sorted = posts_data_sorted.orderBy("year", desc("total_views"))
posts_data_sorted.show(15)

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|               local|      19682|
|2008|            database|      19682|
|2008|                java|      11532|
|2008|         inheritance|      10971|
|2008|           variables|       7700|
|2008|       accessibility|       7700|
|2008|               excel|       6540|
|2008|          automation|       6540|
|2008|           interface|       3271|
|2008|       configuration|       2927|
|2008|dependency-injection|       2927|
|2008|      castle-windsor|       2927|
|2008|               linux|       2393|
+----+--------------------+-----------+
only showing top 15 rows



In [ ]:
#  Определяем спецификацию Window, разбитую по годам и упорядоченную в порядке убывания по показателю total_views
window_spec = Window.partitionBy("year").orderBy(posts_data_sorted["total_views"].desc())

# Вычисляем ранг каждой строки в пределах каждого раздела (года) на основе total_views и добавляем его в новый столбец "rank".
ranked_df = posts_data_sorted.withColumn("rank", rank().over(window_spec))

# Получаем топ-10 тегов по количеству просмотров для каждого года.
result_df = ranked_df.filter(ranked_df["rank"] <= 10)
# Оставляем только нужную информацию для конечного результата.
result_df = result_df.select("year", "tag", "total_views")
#ГРуппируем вывод информации по годам в убывающем порядке
data_sort = result_df.orderBy("year", desc("total_views"))
data_sort.show()


+----+-------------+-----------+
|year|          tag|total_views|
+----+-------------+-----------+
|2008|           c#|      25401|
|2008|         .net|      24321|
|2008|     database|      19682|
|2008|        local|      19682|
|2008|         java|      11532|
|2008|  inheritance|      10971|
|2008|accessibility|       7700|
|2008|    variables|       7700|
|2008|   automation|       6540|
|2008|        excel|       6540|
|2009|           c#|      73661|
|2009|         .net|      39167|
|2009|       python|      32219|
|2009|          c++|      29381|
|2009|     winforms|      25670|
|2009|      asp.net|      25267|
|2009|        loops|      21869|
|2009|     .net-3.5|      20376|
|2009|     pointers|      18702|
|2009|   javascript|      17139|
+----+-------------+-----------+
only showing top 20 rows



In [17]:
# создаем отчет
data_sort.write.parquet("result.parquet")

In [18]:
# Удаление результатов
import shutil
shutil.rmtree("result.parquet")